Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.661480
Minibatch accuracy: 18.8%
Validation accuracy: 12.0%
Minibatch loss at step 50: 2.076706
Minibatch accuracy: 18.8%
Validation accuracy: 40.4%
Minibatch loss at step 100: 1.112260
Minibatch accuracy: 62.5%
Validation accuracy: 64.9%
Minibatch loss at step 150: 0.745046
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.865450
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%
Minibatch loss at step 250: 1.104396
Minibatch accuracy: 68.8%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.458445
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.483440
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 400: 0.431903
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.828215
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.779749
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], 'SAME')
    hidden = tf.nn.relu(pool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], 'SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.514519
Minibatch accuracy: 6.2%
Validation accuracy: 13.8%
Minibatch loss at step 50: 1.709493
Minibatch accuracy: 31.2%
Validation accuracy: 50.5%
Minibatch loss at step 100: 0.942509
Minibatch accuracy: 62.5%
Validation accuracy: 63.2%
Minibatch loss at step 150: 0.406263
Minibatch accuracy: 93.8%
Validation accuracy: 72.2%
Minibatch loss at step 200: 1.161847
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 250: 1.238961
Minibatch accuracy: 56.2%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.344023
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 350: 0.413330
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.286063
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.950424
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.661092
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [14]:
# LeNet 5 notes: http://eblearn.sourceforge.net/beginner_tutorial2_train.html
# LeNet 5 viz: http://eblearn.sourceforge.net/lib/exe/detail.id.mnist_media.mnist_fprop0.png.html?id=mnist&media=mnist_fprop0.png  

batch_size = 16

c1_patch_size = 5
c1_depth = 6
c3_patch_size = 6
c3_depth = 16
c5_patch_size = 6
c5_depth = 120

num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  c1_filters = tf.Variable(tf.truncated_normal(
      [c1_patch_size, c1_patch_size, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_filters = tf.Variable(tf.truncated_normal(
      [c3_patch_size, c3_patch_size, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.zeros([c3_depth]))
  c5_filters = tf.Variable(tf.truncated_normal(
      [c5_patch_size, c5_patch_size, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.zeros([c5_depth]))
  c6_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * c5_depth, num_hidden], stddev=0.1))
  c6_biases = tf.Variable(tf.zeros([num_hidden]))
  end_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  end_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, c1_filters, [1, 1, 1, 1], padding='SAME')
    c1_result = tf.nn.tanh(conv + c1_biases)
    
    s1_result = tf.nn.avg_pool(c1_result, [1,2,2,1], [1,2,2,1], 'SAME')
    conv = tf.nn.conv2d(s1_result, c3_filters, [1, 1, 1, 1], padding='SAME')
    c3_result = tf.nn.tanh(conv + c3_biases)
    
    s4_result = tf.nn.avg_pool(c3_result, [1,2,2,1], [1,2,2,1], 'SAME')
    conv = tf.nn.conv2d(s4_result, c5_filters, [1, 1, 1, 1], padding='SAME')
    c5_result = tf.nn.tanh(conv + c5_biases)
    
    shape = c5_result.get_shape().as_list()
    reshape = tf.reshape(c5_result, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, c6_weights) + c6_biases)
    return tf.matmul(hidden, end_weights) + end_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.839330
Minibatch accuracy: 6.2%
Validation accuracy: 17.2%
Minibatch loss at step 50: 1.169614
Minibatch accuracy: 56.2%
Validation accuracy: 74.5%
Minibatch loss at step 100: 0.863493
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 150: 0.232463
Minibatch accuracy: 93.8%
Validation accuracy: 77.2%
Minibatch loss at step 200: 1.151611
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%
Minibatch loss at step 250: 1.108173
Minibatch accuracy: 68.8%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.308186
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 350: 0.607465
Minibatch accuracy: 93.8%
Validation accuracy: 78.3%
Minibatch loss at step 400: 0.376984
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.890479
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 500: 0.687421
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Mi

In [6]:
# Improved LeNet 5
# added regularization on all weights


batch_size = 128
beta = 0.005

c1_patch_size = 5
c1_depth = 6
c3_patch_size = 6
c3_depth = 16
c5_patch_size = 6
c5_depth = 120

num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  c1_filters = tf.Variable(tf.truncated_normal(
      [c1_patch_size, c1_patch_size, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_filters = tf.Variable(tf.truncated_normal(
      [c3_patch_size, c3_patch_size, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.zeros([c3_depth]))
  c5_filters = tf.Variable(tf.truncated_normal(
      [c5_patch_size, c5_patch_size, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.zeros([c5_depth]))
  c6_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * c5_depth, num_hidden], stddev=0.1))
  c6_biases = tf.Variable(tf.zeros([num_hidden]))
  end_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  end_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, c1_filters, [1, 1, 1, 1], padding='SAME')
    c1_result = tf.nn.relu(conv + c1_biases)
    
    s1_result = tf.nn.avg_pool(c1_result, [1,2,2,1], [1,2,2,1], 'SAME')
    conv = tf.nn.conv2d(s1_result, c3_filters, [1, 1, 1, 1], padding='SAME')
    c3_result = tf.nn.relu(conv + c3_biases)
    
    s4_result = tf.nn.avg_pool(c3_result, [1,2,2,1], [1,2,2,1], 'SAME')
    conv = tf.nn.conv2d(s4_result, c5_filters, [1, 1, 1, 1], padding='SAME')
    c5_result = tf.nn.relu(conv + c5_biases)
    
    shape = c5_result.get_shape().as_list()
    reshape = tf.reshape(c5_result, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, c6_weights) + c6_biases)
    return tf.matmul(hidden, end_weights) + end_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 0.5*beta*tf.nn.l2_loss(c6_weights) + 0.5*beta*tf.nn.l2_loss(end_weights)
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.15, global_step, 3500, 0.86, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.019392
Minibatch accuracy: 8.6%
Validation accuracy: 10.8%
Minibatch loss at step 50: 4.107642
Minibatch accuracy: 82.8%
Validation accuracy: 77.2%
Minibatch loss at step 100: 4.040259
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 150: 4.025841
Minibatch accuracy: 73.4%
Validation accuracy: 82.2%
Minibatch loss at step 200: 3.730654
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Minibatch loss at step 250: 3.428119
Minibatch accuracy: 89.8%
Validation accuracy: 83.8%
Minibatch loss at step 300: 3.557109
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 350: 3.175333
Minibatch accuracy: 88.3%
Validation accuracy: 84.9%
Minibatch loss at step 400: 3.215962
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 450: 3.019794
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Minibatch loss at step 500: 2.847875
Minibatch accuracy: 90.6%
Validation accuracy: 86.1%
Mi